### Initialize Environment and Populate Blockchain Database

In [1]:
import os
import sys
from pathlib import Path

container_src_path = Path('/app/src/')
local_src_path = Path(Path.cwd(), 'src/')

# see if this src path exists.
# if it does, we are in a container.
# if not, we are in local.
if not container_src_path.exists():
    src_path = local_src_path
else:
    src_path = container_src_path

src_path_str = str(src_path)
if src_path_str not in sys.path:
    sys.path.insert(0, src_path_str)

from dotenv import load_dotenv

from blockchain_data_provider import (
    PersistentBlockchainAPIData,
    BlockchainAPIJSON
)

from models.base import SessionLocal, DATABASE_URL
from models.bitcoin_data import Block

# see if database tables exist. if not, create them
from models import base
from sqlalchemy import inspect
from sqlalchemy.sql import text

# block_data_url = "https://blockchain.info/rawblock/{height}?format=json"
block_data_url = "http://localhost:8500/block/{height}"

DELETE_DB = False
DELETE_DB = True

inspector = inspect(base.engine)

if DELETE_DB:
# wipe the database
    with SessionLocal() as session:
        if inspector.has_table("inputs"):
            session.execute(text('DELETE FROM inputs'))
        if inspector.has_table("outputs"):
            session.execute(text('DELETE FROM outputs'))
        if inspector.has_table("transactions"):
            session.execute(text('DELETE FROM transactions'))
        if inspector.has_table("blocks"):
            session.execute(text('DELETE FROM blocks'))
        if inspector.has_table("addresses"):
            session.execute(text('DELETE FROM addresses'))
        session.commit()

    # if Path(Path.cwd(), DATABASE_URL).exists():
    #     os.remove(Path(Path.cwd(), DATABASE_URL))
        
    print("Database wiped.")

if not inspector.has_table("blocks"):
    base.Base.metadata.create_all(bind=base.engine)
    print("No data found. Database created.")

load_dotenv()


Database wiped.
No data found. Database created.


True

In [2]:
# max_height = 100_000
max_height = 100_000

with SessionLocal() as session:
    highest_block = session.query(Block).order_by(Block.height.desc()).first()

if highest_block is not None:
    print(f"highest block: {highest_block.height}")

slow_provider = BlockchainAPIJSON(block_endpoint=block_data_url)
provider = PersistentBlockchainAPIData(data_provider=slow_provider)
with SessionLocal() as session:
    provider.populate_blocks(session, range(0,max_height+1), show_progressbar=True)

with SessionLocal() as session:
    # ensure we can get block 170
    assert len(provider.get_block(session, 170).transactions) == 2
    assert len(provider.get_block(session, 546).transactions) == 4

  0%|          | 0/100001 [00:00<?, ?it/s]

100%|██████████| 100001/100001 [52:16<00:00, 31.88it/s] 


### Get some Data

In [7]:
from blockchain_data_provider import PersistentBlockchainAPIData

slow_provider = BlockchainAPIJSON(block_endpoint=block_data_url)
provider = PersistentBlockchainAPIData(data_provider=slow_provider)

# provider.latest_parsed_block = 2811
# with SessionLocal() as session:
#     provider.populate_block(session, 2812)

data_provider = PersistentBlockchainAPIData()
with SessionLocal() as session:
    block_170 = data_provider.get_block(session, 170)

print(f"transaction count: {len(block_170.transactions)}")

tx_12cb = block_170.transactions[1]
print(f"address of first output: {tx_12cb.outputs[0].address.addr}")
assert provider.get_output(session, 170).value == 5000000000

with SessionLocal() as session:
    block_170 = data_provider.get_block(session, 0)

assert block_170.height == 0
assert block_170.transactions[0].outputs[0].id == 0, block_170.transactions[0].outputs[0].id

transaction count: 2
address of first output: 1PSSGeFHDnKNxiEyFrD1wcEaHr9hrQDDWc


In [ ]:
from sqlalchemy.orm import joinedload

from models.bitcoin_data import Tx, Input, Output

tx_index = 3003916113328251

with SessionLocal() as session:
    tx_obj = session.query(Tx).options(
            joinedload(Tx.inputs)
            .joinedload(Input.prev_out),
            joinedload(Tx.outputs)
            .joinedload(Output.address)    
        ).filter_by(index=tx_index).first()

for out in tx_obj.outputs:
    print(out.address_addr)

1DZTzaBHUDM7T3QvUKBz4qXMRpkg8jsfB5
1DCbY2GYVaAMCBpuBNN5GVg3a47pNK1wdi
